In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from datasets import Dataset, DatasetDict, load_dataset

In [8]:
import random

import numpy as np


def set_seed(seed):
    np.random.seed(seed % (2**32 - 1))
    random.seed(seed)

SEED = 89
set_seed(SEED)

In [11]:
hf_username = 'bdsaglam'

In [13]:
dataset_path = f"{hf_username}/web_nlg-erx-concat"
dataset_name = "release_v3.0_en"
dsd = load_dataset(dataset_path, dataset_name, trust_remote_code=True)

In [14]:
dsd['train'][0]

{'eid': 'Id62',
 'text': 'Adonis Georgiadis is a member of the Greek New Democracy Party which uses the colour blue. Before being succeeded by Makis Voridis he served in office while Antonis Samaras was Prime Minister and Evangelos Venizelos was Deputy.',
 'triples': ['Adonis Georgiadis | in office while prime minister | Antonis Samaras',
  'Antonis Samaras | has deputy | Evangelos Venizelos',
  'New Democracy (Greece) | colour | Blue',
  'Adonis Georgiadis | party | New Democracy (Greece)',
  'Adonis Georgiadis | successor | Makis Voridis']}

In [15]:
def make_openai_chat(example):
    triples_str = "\n".join(example["triples"])
    return {
        "messages": [
            {"role": "user", "content": example["text"]},
            {"role": "assistant", "content": triples_str},
        ],
    }

In [17]:
make_openai_chat(dsd['train'][0])

{'messages': [{'role': 'user',
   'content': 'Adonis Georgiadis is a member of the Greek New Democracy Party which uses the colour blue. Before being succeeded by Makis Voridis he served in office while Antonis Samaras was Prime Minister and Evangelos Venizelos was Deputy.'},
  {'role': 'assistant',
   'content': 'Adonis Georgiadis | in office while prime minister | Antonis Samaras\nAntonis Samaras | has deputy | Evangelos Venizelos\nNew Democracy (Greece) | colour | Blue\nAdonis Georgiadis | party | New Democracy (Greece)\nAdonis Georgiadis | successor | Makis Voridis'}]}

In [20]:
chat_dsd = dsd.map(make_openai_chat, remove_columns=['text', 'triples'])

Map:   0%|          | 0/8871 [00:00<?, ? examples/s]

Map:   0%|          | 0/1116 [00:00<?, ? examples/s]

Map:   0%|          | 0/1834 [00:00<?, ? examples/s]

In [22]:
assert 'eid' in chat_dsd['train'].features
assert 'messages' in chat_dsd['train'].features
chat_dsd['train'][0]

{'eid': 'Id62',
 'messages': [{'content': 'Adonis Georgiadis is a member of the Greek New Democracy Party which uses the colour blue. Before being succeeded by Makis Voridis he served in office while Antonis Samaras was Prime Minister and Evangelos Venizelos was Deputy.',
   'role': 'user'},
  {'content': 'Adonis Georgiadis | in office while prime minister | Antonis Samaras\nAntonis Samaras | has deputy | Evangelos Venizelos\nNew Democracy (Greece) | colour | Blue\nAdonis Georgiadis | party | New Democracy (Greece)\nAdonis Georgiadis | successor | Makis Voridis',
   'role': 'assistant'}]}

In [23]:
new_dataset_path = f"{hf_username}/{dataset_path.split('/', 1)[-1]}-chat"
chat_dsd.push_to_hub(new_dataset_path, config_name=dataset_name)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/bdsaglam/web_nlg-erx-concat-chat/commit/59ac6e05c711389f4e3293157fc8a10c84585db5', commit_message='Upload dataset', commit_description='', oid='59ac6e05c711389f4e3293157fc8a10c84585db5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/bdsaglam/web_nlg-erx-concat-chat', endpoint='https://huggingface.co', repo_type='dataset', repo_id='bdsaglam/web_nlg-erx-concat-chat'), pr_revision=None, pr_num=None)